In [1]:
import os 
import pandas as pd
import json 

from collections import defaultdict

DOC_TYPE = 'BOE'
DATAFRAME_RESULT_PATH = f'/home/administrator/Vasu/EXTRACTION_WORK/EXTRACTION_RESULTS/{DOC_TYPE}'
DOC_EXTRACTION_MODEL_RESULTS = f'/home/administrator/Vasu/EXTRACTION_WORK/EXTRACTION_RESULTS/{DOC_TYPE}/MODEL_RESULTS'
DOC_EXTRACTION_POST_PROCESSING_RESULTS = f'/home/administrator/Vasu/EXTRACTION_WORK/EXTRACTION_RESULTS/{DOC_TYPE}/POST_PROCESSING_RESULTS'

In [2]:
df = pd.DataFrame(columns=['Document_Type','Image_Name','Model_Key','Post_Processing_Key','Model_Address','Post_Processing_Country'])
model_mapping_dict = defaultdict(set)
postprocessing_mapping_dict = defaultdict(set)
final_mapping_dict = defaultdict(list)
connected_key_dict = {
    "drawee_bank_address":"drawee_bank_country",
    'drawee_address':"drawee_country",
    'from_place':'from_place',
    'to_place':'to_place' 
}

In [3]:
for file in os.listdir(DOC_EXTRACTION_MODEL_RESULTS):
    file_name = file[:-18]
    file_path = os.path.join(DOC_EXTRACTION_MODEL_RESULTS,file)
    with open(file_path,'r') as f:
        try:
            data = json.load(f)             
            for key in data.keys():
                page_data = data[key]
                for page_wise_key in page_data.keys():
                    if 'place' in page_wise_key or 'address' in page_wise_key:
                        if page_data[page_wise_key]:
                            potential_value_lst = page_data[page_wise_key]
                            for lst in potential_value_lst:
                                model_mapping_dict[file_name].add((DOC_TYPE,page_wise_key,lst[0]))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {file_path}: {e}")
print(model_mapping_dict)

defaultdict(<class 'set'>, {'pdf16.pdf_5.png': {('BOE', 'issue_place', 'seoul , korea'), ('BOE', 'drawee_bank_address', 'sco 104-105 , district centre , sector 16 , faridabad - 121007')}, 'pdf28-16.png': {('BOE', 'drawee_address', '6-7,1 || to 1 | 8 , ellaya mudali street , tondiarpet , chennai - 600 081 india उन'), ('BOE', 'drawee_address', 'g - 7,1 / 1 to 1 / 8 , ella ya mudali street , tondiarpet , chennai - 600 081 india')}, 'pdf36_page-0004.jpg': {('BOE', 'drawee_address', 'p.o. box 29228 addis ababa ethiopia'), ('BOE', 'drawer_address', 'p.o box : 3202 a / 64b , nizamuddin east new delhi 110013 india'), ('BOE', 'drawer_bank_address', 'johannesburg airport , south africa addis ababa airport , ethiopia .'), ('BOE', 'drawee_bank_address', 'megenagna branch p.o. box 12638 addis ababa ethiopia'), ('BOE', 'drawer_bank_address', '9 - a , connaught place , new delhi - 110001 , india')}, 'pdf30-7.png': {('BOE', 'drawee_address', 's - 40 , 1 st floor , janta market , rajouri garden , new d

In [4]:
for file in os.listdir(DOC_EXTRACTION_POST_PROCESSING_RESULTS):
    file_name = file[:-28]
    file_path = os.path.join(DOC_EXTRACTION_POST_PROCESSING_RESULTS,file)
    with open(file_path,'r') as f:
        try:
            data = json.load(f)
            for key in data['keys_extraction'].keys():
                if 'country' in key or 'name' in key:
                    if data['keys_extraction'][key] and 'value' in  data['keys_extraction'][key].keys():
                        if data['keys_extraction'][key]['value'] != '':
                            postprocessing_mapping_dict[file_name].add((DOC_TYPE,key,data['keys_extraction'][key]['value']))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {file_path}: {e}")
print(postprocessing_mapping_dict)

defaultdict(<class 'set'>, {'pdf33-6.png': {('BOE', 'country_of_origin', 'INDONESIA'), ('BOE', 'drawer_bank_name', 'PT. BANK WOORI INDONESIA~ICICI IC/C/BANK LTD. BANK LIMITED'), ('BOE', 'drawee_name', 'MAHARASHTRA FEEDS PRIVATE LTD'), ('BOE', 'drawer_bank_country', 'INDONESIA')}, 'pdf9.pdf_9.png': {('BOE', 'drawee_bank_country', 'INDIA'), ('BOE', 'drawee_bank_name', 'ICICI BANK LTD'), ('BOE', 'drawer_bank_name', 'NATIONAL BANK OF'), ('BOE', 'drawee_name', 'DSG PAPER PVT. LTD'), ('BOE', 'country_of_origin', 'INDIA'), ('BOE', 'drawee_country', 'INDIA'), ('BOE', 'drawer_name', 'ZENATH')}, 'pdf10.pdf_7.png': {('BOE', 'drawer_bank_name', 'ORIENTAL BANK OF COMMERCE~LTD BANK ICICI'), ('BOE', 'drawee_bank_name', 'ICICI BANK'), ('BOE', 'country_of_origin', 'INDIA'), ('BOE', 'drawee_country', 'INDIA'), ('BOE', 'drawer_name', 'LEITNER SHRIRAM MANUFACTURING LTD.'), ('BOE', 'drawer_bank_country', 'INDIA')}, 'pdf35-08.png': {('BOE', 'drawee_bank_name', 'ICICI BANK LIMITED~ICICI BANK'), ('BOE', 'draw

In [5]:
visited_keys = set()
for key in model_mapping_dict.keys():
    model_set = model_mapping_dict[key]
    for document_type, field_key, value in model_set:
        if field_key in connected_key_dict.keys():
            post_processing_set = postprocessing_mapping_dict[key]
            target_key = connected_key_dict[field_key]         
            found = False
            for elem in post_processing_set:
                if elem[1] == target_key:
                    final_mapping_dict[key].append((document_type, key, field_key, elem[1], value, elem[2]))
                    found = True
                    visited_keys.add(target_key)
                    break 
            if not found:
                final_mapping_dict[key].append((document_type, key, field_key, '', value, ''))
        else:
            final_mapping_dict[key].append((document_type, key, field_key, '', value, ''))

for key in postprocessing_mapping_dict.keys():
    post_processing_set = postprocessing_mapping_dict[key]
    for document_type, field_key, value in post_processing_set:
        if field_key not in visited_keys and field_key in connected_key_dict.values():
            final_mapping_dict[key].append((document_type, key, '', field_key, '', value))

In [6]:
rows = []
for key in final_mapping_dict.keys():
    result = final_mapping_dict[key]
    for res in result:
        document_type,key,model_key,post_processing_key,model_value,post_processing_value = res
        rows.append({
            'Document_Type':document_type,
            'Image_Name':key,
            'Model_Key':model_key,
            'Post_Processing_Key':post_processing_key,
            'Model_Address':model_value,
            'Post_Processing_Country':post_processing_value
        })
df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)
df.drop_duplicates()
csv_path = DATAFRAME_RESULT_PATH + f'/{DOC_TYPE}_Countries.csv'
df.to_csv(csv_path)